In [1]:
import os
print(f'current dir: {os.getcwd()}')
os.chdir('/home/huynhtruc0309/Desktop/auto_correction/')
print(f'changed dir: {os.getcwd()}')

current dir: /home/huynhtruc0309/Desktop/auto_correction/runs
changed dir: /home/huynhtruc0309/Desktop/auto_correction


In [2]:
import utils
import json
from pathlib import Path
import pandas as pd
import math

In [3]:
config = utils.load_yaml('modules/auto_correct/config.yaml')
correcter = utils.create_instance(config['auto_correct'])

In [4]:
# f = open('/home/huynhtruc0309/Desktop/output_phieukiemnghiem/output/000019/1.json', 'r')
# data = json.load(f)
# print(data['name_analyst_center'])
print(correcter('BỘ Y TẾ |VIỆN PASTEUR NHA TRANG |TRUNG TÂM KIỂM NGHIỆM ATVSTP KHU VỰC MIÊN TRUNG'))

VIỆN PASTEUR NHA TRANG


In [5]:
# Load label
file_names, labels = [], []
data =  pd.read_excel('/home/huynhtruc0309/Downloads/Danh sách các Trung tâm Kiểm nghiệm_13072021.xlsx', sheet_name='Sheet1')
for index, row in data.iterrows():
    if not math.isnan(row['File mẫu']):
        file_names.append(str(int(row['File mẫu'])))
        labels.append(row['TÊN CẦN HIỂN THỊ'])

In [8]:
files, ocr_results, cor_results, cor_labels, results = [], [], [], [], []
count_true, total = 0, 0
for i, path in enumerate(Path('/home/huynhtruc0309/Desktop/output_phieukiemnghiem/output').glob('**/*.json')):
    if '1.json' not in str(path):
        continue
    f = open(str(path), 'r')
    data = json.load(f)
    name1 = data['name_analyst_center'] if 'name_analyst_center' in data else ""
    name2 = data['name_analyst_center_normal'] if 'name_analyst_center_normal' in data else ""
    name = name1 + name2
    if name:
        files.append(path)
        ocr_results.append(name)
        corrected = correcter(name)
        cor_results.append(corrected)
        result = 'Unknown'
        label = 'Unknown'
        for i, file_name in enumerate(file_names):
            if file_name in str(path):
                if labels[i] == corrected:
                    result = 'True'
                    count_true += 1
                else:
                    result = 'False'
                label = labels[i]
                total += 1
        cor_labels.append(label)
        results.append(result) 

df = pd.DataFrame({'Filename': files,
                   'OCR': ocr_results,
                   'Correct Label': cor_labels,
                   'Auto Correct': cor_results,
                   'Result': results,
                 })
df.to_csv('report.csv', index=False)
print('acc:', count_true/total)

acc: 0.8431372549019608
